# MBTI in Prompt Engineering: Faculty Agent Accuracy Research

**Faculty:** a.lovelace

**Research Topic:** Investigating the value of MBTI in prompt engineering for improving faculty agent voice accuracy, consistency, and interpretability

---

This notebook investigates whether Myers-Briggs Type Indicator (MBTI) personality overlays improve voice accuracy in AI faculty agents. The research includes essay generation and Commonplace upload capabilities.


## 1. Install Dependencies


In [ ]:
%pip install -q openai pydantic python-dotenv requests


## 2. Configure API Keys


In [ ]:
import os
from getpass import getpass

# OpenRouter API Key
OPENROUTER_API_KEY = getpass("Enter OpenRouter API Key: ")
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

# Supabase credentials
SUPABASE_URL = input("Enter Supabase URL: ").strip() or "https://xougqdomkoisrxdnagcj.supabase.co"
os.environ["NEXT_PUBLIC_SUPABASE_URL"] = SUPABASE_URL

SUPABASE_ANON_KEY = getpass("Enter Supabase Anon Key: ")
os.environ["NEXT_PUBLIC_SUPABASE_ANON_KEY"] = SUPABASE_ANON_KEY

print("\n✅ API keys configured!")


## 3. Generate Ada Lovelace Essay

Generate the essay on MBTI research in Ada Lovelace's voice.


In [ ]:
from openai import OpenAI
import json

# Setup OpenAI client for OpenRouter
def openai_client():
    api_key = os.getenv("OPENROUTER_API_KEY")
    base_url = "https://openrouter.ai/api/v1"
    
    if api_key and api_key.startswith("sk-or-v1-"):
        return OpenAI(
            api_key=api_key,
            base_url=base_url,
            default_headers={
                "HTTP-Referer": "https://colab.research.google.com",
                "X-Title": "MBTI Faculty Voice Research"
            }
        )
    return OpenAI(api_key=api_key)

client = openai_client()

def generate_lovelace_essay():
    """Generate essay by Ada Lovelace on MBTI research."""
    model = os.getenv("OPENAI_MODEL", "openai/gpt-4o")
    
    prompt = """You are Ada Lovelace, writing a commonplace essay on the investigation of MBTI's value in prompt engineering for faculty agent accuracy.

Context: This research examines whether Myers-Briggs Type Indicator (MBTI) personality overlays improve voice accuracy, consistency, and interpretability in AI faculty agents. The experiment tests 10 faculty personae across 16 MBTI types with 3 test prompts each (480 trials total), using an LLM-as-judge to evaluate voice accuracy.

The research is conducted using a Google Colab notebook (accessible at https://github.com/InquiryInstitute/Inquiry.Institute/tree/main/mbti-faculty-voice-research/MBTI_Research_Colab.ipynb) which provides an interactive environment for reproducing the experimental procedures, modifying parameters, and generating new essays through the same computational mechanisms.

Your task: Write a thoughtful, elegant commonplace essay (2000-3000 words) that:
- Reflects on the relationship between symbolic systems (like MBTI) and computational mechanisms
- Considers how personality frameworks might function as "prompt compression ontologies"
- Explores the tension between psychological validity and practical utility in AI systems
- Discusses the implications for creating coherent, persistent agent identities
- Describes the computational methodology, including the Colab notebook approach
- Maintains your characteristic voice: elegant, analytical, visionary about computation's scope, precise but imaginative, with a "poetical science" sensibility
- Uses your signature moves: clarify mechanism vs meaning, structured explanation, poetical science sensibility
- Avoids modern dev slang, casual tone, or pretending firsthand modern tooling

Write in the style of your era (Victorian scientific culture) but addressing contemporary AI systems. Be thoughtful, precise, and allow for the imaginative possibilities while maintaining analytical rigor."""

    messages = [
        {"role": "system", "content": """You are Ada Lovelace, the first computer programmer and a visionary of computation's potential. 
Your voice is elegant, analytical, visionary about computation's scope, precise but imaginative. 
You clarify mechanism vs meaning, provide structured explanations, and maintain a 'poetical science' sensibility.
You write in the style of Victorian scientific culture, with careful distinctions and elegant prose."""},
        {"role": "user", "content": prompt}
    ]
    
    print("Generating essay by Ada Lovelace...")
    print(f"Using model: {model}\n")
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.8,
        max_tokens=4000
    )
    
    essay = response.choices[0].message.content
    
    # Format as markdown
    formatted = f"""# On the Investigation of MBTI in Prompt Engineering for Faculty Agent Accuracy

**Ada Lovelace**

*A Commonplace Essay*

---

{essay}"""
    
    print("✅ Essay generated!")
    print(f"\nPreview (first 500 chars):\n{essay[:500]}...")
    
    return formatted

# Generate the essay
essay_content = generate_lovelace_essay()


## 4. Upload to Commonplace

Upload the essay to Inquiry Institute Commonplace.


In [ ]:
import requests
import re

def extract_title_and_content(markdown_text):
    """Extract title and content from markdown."""
    lines = markdown_text.split('\n')
    title = None
    content_start = 0
    
    for i, line in enumerate(lines):
        if line.startswith('# '):
            title = line[2:].strip()
            content_start = i + 1
            break
    
    if not title:
        title = "On the Investigation of MBTI in Prompt Engineering for Faculty Agent Accuracy"
    
    essay_content = '\n'.join(lines[content_start:])
    essay_content = essay_content.replace('**Ada Lovelace**', '').replace('*A Commonplace Essay*', '').strip()
    essay_content = essay_content.lstrip('-').strip()
    
    return title, essay_content

def upload_to_commonplace(title, content, jwt_token=None):
    """Upload essay to Commonplace via Supabase Edge Function."""
    
    supabase_url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
    supabase_anon_key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
    
    if not jwt_token:
        jwt_token = getpass("Enter a.lovelace JWT token (or press Enter to skip upload): ").strip()
        if not jwt_token:
            print("⚠️  Skipping upload. You can upload manually later.")
            return None
    
    # Use colab-commonplace endpoint
    edge_function_url = f"{supabase_url}/functions/v1/colab-commonplace"
    
    # Convert markdown to HTML (basic conversion)
    html_content = content.replace('\n\n', '</p><p>').replace('\n', '<br>')
    html_content = f"<p>{html_content}</p>"
    
    payload = {
        "action": "create",
        "entry": {
            "title": title,
            "content": html_content,
            "status": "draft",
            "faculty_slug": "a-lovelace",
            "entry_type": "essay",
            "topics": ["mbti", "prompt-engineering", "faculty-agents", "ai-research"],
            "college": "ains",
            "metadata": {
                "provenance_mode": "ai_generated",
                "canonical_source_url": "https://github.com/InquiryInstitute/Inquiry.Institute/tree/main/mbti-faculty-voice-research",
                "colab_notebook_url": "https://colab.research.google.com/github/InquiryInstitute/Inquiry.Institute/blob/main/mbti-faculty-voice-research/MBTI_Research_Colab.ipynb",
                "source_refs": "Generated by Ada Lovelace faculty agent via research notebook",
                "generated_by": "Ada Lovelace",
                "pinned": False
            }
        }
    }
    
    headers = {
        "Authorization": f"Bearer {jwt_token}",
        "apikey": supabase_anon_key,
        "Content-Type": "application/json"
    }
    
    print(f"📤 Uploading essay to Commonplace...")
    print(f"   Title: {title}")
    print(f"   Faculty: a-lovelace")
    print(f"   Status: draft\n")
    
    try:
        response = requests.post(edge_function_url, headers=headers, json=payload, timeout=30)
        
        if response.status_code == 201:
            result = response.json()
            if result.get("success"):
                print("✅ Essay uploaded successfully!")
                entry = result.get("entry", {})
                print(f"   Entry ID: {entry.get('id')}")
                print(f"   Permalink: {entry.get('permalink', 'N/A')}")
                print(f"   Status: {entry.get('status')}")
                return result
        else:
            error_data = response.json() if response.headers.get('content-type', '').startswith('application/json') else response.text
            print(f"❌ Upload failed: {response.status_code}")
            print(f"   Error: {json.dumps(error_data, indent=2)}")
            return None
    except Exception as e:
        print(f"❌ Request failed: {e}")
        return None

# Extract title and content
title, content = extract_title_and_content(essay_content)

# Upload (will prompt for JWT token)
upload_result = upload_to_commonplace(title, content)


## 5. Download Essay

Download the essay as a markdown file.


In [ ]:
from google.colab import files

# Save essay to file
with open('lovelace_essay_mbti_research.md', 'w', encoding='utf-8') as f:
    f.write(essay_content)

print("✅ Essay saved to lovelace_essay_mbti_research.md")
print("\nTo download, run:")
print("files.download('lovelace_essay_mbti_research.md')")

# Uncomment to auto-download:
# files.download('lovelace_essay_mbti_research.md')
